# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [14]:
import pandas as pd
import numpy as np

# Cleaning function without inplace warnings
def clean_data(df):
    # Standardize column names
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    
    # Dropping duplicates
    df = df.drop_duplicates()
    
    # Filling missing numeric columns with median
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        if df[col].isnull().sum() > 0:
            df.loc[:, col] = df[col].fillna(df[col].median())
    
    # Fill missing categorical columns with mode
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isnull().sum() > 0:
            df.loc[:, col] = df[col].fillna(df[col].mode()[0])
    
    return df

# Reading the three files
file1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv")
file2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")
file3 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv")

# Clean all three datasets
clean_file1 = clean_data(file1)
clean_file2 = clean_data(file2)
clean_file3 = clean_data(file3)

# Combine them into one DataFrame
combined_df = pd.concat([clean_file1, clean_file2, clean_file3], ignore_index=True)

# Drop duplicates in combined data
combined_df = combined_df.drop_duplicates()

# Save cleaned combined data to CSV
combined_df.to_csv("combined_cleaned_data.csv", index=False)

# Quick validation prints
print(f"Shape of combined cleaned dataset: {combined_df.shape}")
print(f"Null values remaining: \n{combined_df.isnull().sum()}")
print(f"Any duplicates left? {combined_df.duplicated().any()}")


Shape of combined cleaned dataset: (9135, 12)
Null values remaining: 
customer                        0
st                           7070
gender                          0
education                       0
customer_lifetime_value         0
income                          0
monthly_premium_auto            0
number_of_open_complaints       0
policy_type                     0
vehicle_class                   0
total_claim_amount              0
state                        2065
dtype: int64
Any duplicates left? False


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [18]:
# Loading dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

# Basic overview
print("Dataset shape:", df.shape)
print("\nColumns:\n", df.columns)
print("\nFirst 5 rows:\n", df.head())

# Drop unnecessary columns
if 'unnamed:_0' in df.columns:
    df = df.drop(columns=['unnamed:_0'])

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Check for null values
print("\nNull values per column:\n", df.isnull().sum())

# Convert 'effective_to_date' to datetime
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# Explore categorical columns (unique values)
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"\nUnique values in '{col}':")



Dataset shape: (10910, 27)

Columns:
 Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type', 'month'],
      dtype='object')

First 5 rows:
    unnamed:_0 customer       state  customer_lifetime_value response  \
0           0  DK49336     Arizona              4809.216960       No   
1           1  KX64629  California              2228.525238       No   
2           2  LZ68649  Washington             14947.917300       No   
3           3  XL78013      Oregon             22332.439460      Yes   
4           4  QA50777      Oregon 

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [20]:
# Total revenue by sales channel
total_revenue_by_channel = df.pivot_table(
    index='sales_channel',
    values='total_claim_amount',
    aggfunc='sum'
).round(2)

print("Total Revenue by Sales Channel:")
print(total_revenue_by_channel)


# Average customer lifetime value by gender and education
avg_ltv_by_gender_education = df.pivot_table(
    index='gender',
    columns='education',
    values='customer_lifetime_value',
    aggfunc='mean'
).round(2)

print("\nAverage Customer Lifetime Value by Gender and Education:")
print(avg_ltv_by_gender_education)


Total Revenue by Sales Channel:
               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04

Average Customer Lifetime Value by Gender and Education:
education  Bachelor  College   Doctor  High School or Below   Master
gender                                                              
F           7874.27  7748.82  7328.51               8675.22  8157.05
M           7703.60  8052.46  7415.33               8149.69  8168.83


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [22]:
# Grouping by policy type and month to count complaints
complaints_summary = df.groupby(['policy_type', 'month'])['number_of_open_complaints'].sum().reset_index()

# Renaming columns for clarity
complaints_summary.columns = ['Policy Type', 'Month', 'Number of Complaints']

# Showing the long format table
print(complaints_summary)


      Policy Type  Month  Number of Complaints
0  Corporate Auto      1            443.434952
1  Corporate Auto      2            385.208135
2   Personal Auto      1           1727.605722
3   Personal Auto      2           1453.684441
4    Special Auto      1             87.074049
5    Special Auto      2             95.226817
